<a href="https://colab.research.google.com/github/HugoKD/NLP/blob/main/similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal** : Given an anchor, a query, a positive and a negative give a higher similarity score to the positive than to the negative given the query and the anchor

# TODO : Implémenter une version end-to-end différentiable
## -> : inclure tous les calculs dans le graphe de calcul PyTorch pour permettre à la backpropagation
##de remonter à travers toute la chaîne, y compris la génération T5 et l'encodage BERT.
Aujourd'hui, seule la génération T5 est entraînée, et BERT est gelé (utilisé comme fonction de scoring externe).
Dans cette version future, on envisagerait :
   - soit d'intégrer l'encodage BERT au graphe pour éventuellement co-entraîner une fonction de scoring,
   - soit de remplacer l'étape de génération (non différentiable) par un mécanisme differentiable (ex : sampling continu / softmax différentiable),
   - soit d'utiliser une loss de type Reinforcement Learning (ex : REINFORCE ou reward-based learning) pour remonter la qualité d'une séquence générée,     même si elle passe par une étape non différentiable (comme decode + encode + score).

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import tensorflow as tf
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer, losses

import torch
import torch.nn as nn
from sentence_transformers import losses
from transformers import AutoTokenizer, AutoModel

In [ ]:
from tqdm import tqdm
from torch.utils.data import DataLoader

In [ ]:
pip install transformers sentencepiece

## EDA

In [ ]:
print(len(data))

In [ ]:
print(type(data[0]))

In [ ]:
print(data[0].keys())

In [ ]:
for i,t in enumerate(data) :
  if t["anchor"].lower().find('invention') == -1 :
    if t["anchor"].lower().find('TECHINCAL FIELD'.lower()) == -1:
      if t["anchor"].lower().find('TECHNICAL FIELD'.lower()) == -1:
        print(i,t["anchor"].lower().find('Disclosure'.lower()))



In [ ]:
for i, t in enumerate(data): #preference : invention, Technical Field, TECHINCAL FIELD, disclosure,This application
  point = (t["anchor"].lower().find('technical field'),t["anchor"].lower().find('TECHINCAL FIELD'.lower()),t["anchor"].lower().find('invention'))
  if point == (-1,-1,-1):
    print(i,t)

## Prepro

In [ ]:
data = pd.DataFrame(data)

In [ ]:
def truncate(row):
    row_lower = row.lower()
    point = next((p for p in (
        row_lower.find('technical field'),
        row_lower.find('invention'),
        row_lower.find('disclosure'),
        row_lower.find('this application')
    ) if p != -1), None)

    if point is not None:
        return row[point:]
    else:
        print("No match found in:", row)
        return None

data['prepro_anchor'] = data['anchor'].apply(truncate)

In [ ]:
l_prepro,l  = data['prepro_anchor'].apply(lambda x : len(x)),data['anchor'].apply(lambda x : len(x))
plt.figure(figsize=(10, 5))
plt.hist(l_prepro, bins=50, color= 'red')
plt.hist(l, bins=50, color = 'blue')

plt.xlabel("Nombre de mots")
plt.ylabel("Nombre de documents")
plt.grid(True)
plt.show()

In [ ]:
l_prepro,l  = data['positive'].apply(lambda x : len(x)),data['negative'].apply(lambda x : len(x))
plt.figure(figsize=(10, 5))
plt.hist(l_prepro, bins=50, color= 'green')
plt.hist(l, bins=50, color = 'purple')

plt.xlabel("Nombre de mots")
plt.ylabel("Nombre de documents")
plt.grid(True)
plt.show()

## Test avec la concaténation :

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_name = "bert-base-uncased"
bert_tokenizer = BertTokenizer.from_pretrained(model_name)
bert = BertModel.from_pretrained(model_name)

bert = bert.to(device)

In [ ]:
#generate a response of our query given the anchor
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

T5 = model.to(device)

### Some other lil tests

In [ ]:
def get_bert_embedding(text,bert):

    tokens = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    bert.to(device)
    tokens = {key: value.to(device) for key, value in tokens.items()}
    with torch.no_grad():
        outputs = bert(**tokens)

    sentence_embedding = outputs.last_hidden_state[:, 0, :]
    return sentence_embedding.cpu().numpy()

In [ ]:
def generate_T5(row):
  inputs = tokenizer(row, return_tensors="pt", padding=True,max_length=512).to(device)
  output = T5.generate(
      **inputs,
      num_beams=4,
      max_length = 512,
      early_stopping=True
  )
  response = tokenizer.decode(output[0], skip_special_tokens=True)
  print(row)
  return response

In [ ]:
data = data[:20]

In [ ]:
data['inputs'] = "Question : " + data['query'] + 'Context : ' + data['prepro_anchor']

In [ ]:
generate_T5(data.iloc[0]['inputs'])

In [ ]:
data.iloc[0]['query']

In [ ]:
data['respose'] = data['inputs'].apply(generate_T5)

In [ ]:
data.rename(columns={'respose':'response'},inplace=True)

In [ ]:
data['response_embedding'] = data['response'].apply(lambda x : get_bert_embedding(x,bert))

In [ ]:
response_embedding = tf.convert_to_tensor(data['response_embedding'].tolist())
response_embedding = tf.squeeze(response_embedding, axis=1)

positive_embedding = tf.convert_to_tensor(data["positive"])
negative_embedding = tf.convert_to_tensor(data["negative"])

In [ ]:
triplet_loss = losses.TripletLoss(model=model, distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE, margin=0.3)
loss_value = triplet_loss.forward(response_embedding, positive_embedding, negative_embedding)

print("Triplet Loss:", loss_value.item())

By including bert in the training loop, i feel like I’m drifting away from my initial goal, which is to generate a response that’s close to the positive example and far from the negative one. <br>


It feels like I’m artificially lowering the loss, since the model is also learning how to represent those elements better <br>
I was actually thinking of BERT more like an external evaluator, rather than something that’s truly part of the model being optimized.

#### BERT frozen version

In [ ]:
from torch.utils.data import Dataset

class TripletTextDataset(Dataset):
    def __init__(self, dataframe):
        self.anchors = dataframe['inputs'].tolist()
        self.positives = dataframe['positive'].tolist()
        self.negatives = dataframe['negative'].tolist()

    def __len__(self):
        return len(self.anchors)

    def __getitem__(self, idx):
        return {
            'anchor': self.anchors[idx],
            'positive': self.positives[idx],
            'negative': self.negatives[idx]
        }

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import json
import pandas as pd
from transformers import AutoTokenizer, BertModel, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm

class TripletTextDataset(Dataset):
    def __init__(self, dataframe):
        self.anchors = dataframe['inputs'].tolist()
        self.positives = dataframe['positive'].tolist()
        self.negatives = dataframe['negative'].tolist()

    def __len__(self):
        return len(self.anchors)

    def __getitem__(self, idx):
        return {
            'anchor': self.anchors[idx],
            'positive': self.positives[idx],
            'negative': self.negatives[idx]
        }

class CustomModel:
    def __init__(self, truncate=True, path='/content/dataset_big_patent_v3.json',
                 bert_model_name="bert-base-uncased", margin=0.3, device=None):

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device

        self.tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.bert = BertModel.from_pretrained(bert_model_name).to(self.device)

        self.t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
        self.T5 = T5ForConditionalGeneration.from_pretrained("t5-small").to(self.device)

        self.triplet_loss_fn = torch.nn.TripletMarginLoss(margin=margin, p=2)

        self.load_data(path)
        if truncate:
            self.data["anchor"] = self.data["anchor"].apply(self.truncate_text)

        self.data['inputs'] = 'question: ' + self.data['query'] + '\ncontext: ' + self.data['anchor']
        self.dataset = TripletTextDataset(self.data)

    def load_data(self, path):
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        self.data = pd.DataFrame(data)

    def truncate_text(self, text):
        text_lower = text.lower()
        point = next((p for p in (
            text_lower.find('technical field'),
            text_lower.find('invention'),
            text_lower.find('disclosure'),
            text_lower.find('this application')
        ) if p != -1), None)
        return text[point:] if point is not None else text

    def get_embedding(self, text, requires_grad=False):
        tokens = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(self.device)
        if requires_grad:
            outputs = self.bert(**tokens)
        else:
            with torch.no_grad():
                outputs = self.bert(**tokens)
        return outputs.last_hidden_state[:, 0, :].squeeze(0)

    def fit(self, optimizer, batch_size=8, epochs=10):
        self.T5.train()
        self.bert.eval()
        dataloader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            total_loss = 0.0
            print(f"\nEpoch {epoch + 1}/{epochs}")

            for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
                input_texts = batch['anchor']
                tokenized = self.t5_tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to(self.device)
                generated_ids = self.T5.generate(
                    **tokenized,
                    num_beams=4,
                    max_length=256,
                    early_stopping=True
                )
                generated_texts = [self.t5_tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

                # Get anchor embeddings with grad
                anchor_emb = torch.stack([self.get_embedding(text, requires_grad=True) for text in generated_texts])

                # Get positive and negative embeddings without grad
                positive_emb = torch.stack([self.get_embedding(p) for p in batch['positive']])
                negative_emb = torch.stack([self.get_embedding(n) for n in batch['negative']])

                optimizer.zero_grad()
                loss = self.triplet_loss_fn(anchor_emb, positive_emb, negative_emb)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            mean_loss = total_loss / len(dataloader)
            print(f"Mean triplet loss for epoch {epoch + 1}: {mean_loss:.4f}")

        return mean_loss

In [ ]:
model = CustomModel(path='/content/dataset_big_patent_v3.json')
optimizer = torch.optim.AdamW(model.T5.parameters(), lr=5e-5)

In [ ]:
mtorch.cuda.empty_cache()

In [ ]:
model.fit(optimizer)

### Training T5 + BERT

Rq : .detach().cpu(). peut être utile pour stocker des éléments autre part que dans le gpu, et donc économiser de la mémoire pour des calculs vraiement utiles.

In [ ]:
from torch.utils.data import Dataset

class TripletTextDataset(Dataset):
    def __init__(self, dataframe):
        self.anchors = dataframe['inputs'].tolist()
        self.positives = dataframe['positive'].tolist()
        self.negatives = dataframe['negative'].tolist()

    def __len__(self):
        return len(self.anchors)

    def __getitem__(self, idx):
        return {
            'anchor': self.anchors[idx],
            'positive': self.positives[idx],
            'negative': self.negatives[idx]
        }

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics import accuracy_score
import json
import pandas as pd
from tqdm import tqdm

class CustomModel(nn.Module):
    def __init__(self, truncate=True, path='/content/dataset_big_patent_v3.json',
                 bert_model_name="bert-base-uncased", margin=0.3, device=None):
        super().__init__()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") if device is None else device

        self.tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
        self.bert = AutoModel.from_pretrained(bert_model_name).to(self.device)

        self.t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
        self.T5 = T5ForConditionalGeneration.from_pretrained("t5-small").to(self.device)

        self.triplet_loss_fn = torch.nn.TripletMarginLoss(margin=margin, p=2)

        self.load_data(path)
        if truncate:
            self.data["anchor"] = self.data["anchor"].apply(self.truncate_text)

        self.data['inputs'] = 'question: ' + self.data['query'] + '\ncontext: ' + self.data['anchor']
        self.dataset = TripletTextDataset(self.data)

    def load_data(self, path):
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        self.data = pd.DataFrame(data)

    def truncate_text(self, text):
        text_lower = text.lower()
        point = next((p for p in (
            text_lower.find('technical field'),
            text_lower.find('invention'),
            text_lower.find('disclosure'),
            text_lower.find('this application')
        ) if p != -1), None)
        return text[point:] if point is not None else text

    def get_embedding(self, text, requires_grad=True):
        tokens = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(self.device)
        if requires_grad:
            outputs = self.bert(**tokens)
        else:
            with torch.no_grad():
                outputs = self.bert(**tokens)
        return outputs.last_hidden_state[:, 0, :].squeeze(0)

    def fit(self, optimizer, batch_size=32, epochs=20):
        self.T5.train()
        self.bert.train()  # Enable training for BERT

        dataloader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True)

        for epoch in range(epochs):
            total_loss = 0.0
            print(f"\nEpoch {epoch + 1}/{epochs}")

            for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
                input_texts = batch['anchor']
                tokenized = self.t5_tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to(self.device)
                generated_ids = self.T5.generate(
                    **tokenized,
                    num_beams=4,
                    max_length=512,
                    early_stopping=True
                )
                generated_texts = [self.t5_tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

                # Get anchor embeddings with grad
                anchor_emb = torch.stack([self.get_embedding(text, requires_grad=True) for text in generated_texts])

                # Get positive and negative embeddings without grad
                positive_emb = torch.stack([self.get_embedding(p) for p in batch['positive']])
                negative_emb = torch.stack([self.get_embedding(n) for n in batch['negative']])

                optimizer.zero_grad()
                loss = self.triplet_loss_fn(anchor_emb, positive_emb, negative_emb)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()


                del tokenized, generated_ids, generated_texts
                del anchor_emb, positive_emb, negative_emb, loss
                torch.cuda.empty_cache()



            mean_loss = total_loss / len(dataloader)
            print(f"Mean triplet loss for epoch {epoch + 1}: {mean_loss:.4f}")

        return mean_loss

In [ ]:
# Example usage:
model = CustomModel()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
model.fit(optimizer, batch_size=12, epochs=10)

In [ ]:
'''torch.save(model.T5.state_dict(), "t5_checkpoint_similarity_4elmts.pt")
torch.save({
    'epoch': 10,
    'model_state_dict': model.T5.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 't5_checkpoint_full_similarity_4elmts.pt')'''

In [ ]:
print(f"Memory used: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

Evaluate it (on the train dataset too ... )

In [ ]:
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
import numpy as np

def evaluate(model, batch_size=16, epochs=1):
    model.T5.eval()
    model.bert.eval()
    model.to(model.device)

    dataloader = DataLoader(model.dataset, batch_size=batch_size, shuffle=True)
    correct = 0

    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}"):
            with torch.no_grad():
                input_texts = batch['anchor']
                tokenized = model.t5_tokenizer(
                    input_texts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True
                ).to(model.device)

                generated_ids = model.T5.generate(
                    **tokenized,
                    num_beams=4,
                    max_length=512,
                    early_stopping=True
                )

                generated_texts = [model.t5_tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]

                anchor_emb = torch.stack([
                    model.get_embedding(text).to(model.device)
                    for text in generated_texts
                ])

                positive_emb = torch.stack([
                    model.get_embedding(p).to(model.device)
                    for p in batch['positive']
                ])
                negative_emb = torch.stack([
                    model.get_embedding(n).to(model.device)
                    for n in batch['negative']
                ])

                # cosine_similarity shape: [batch_size]
                sim_pos = torch.nn.functional.cosine_similarity(anchor_emb, positive_emb, dim=1)
                sim_neg = torch.nn.functional.cosine_similarity(anchor_emb, negative_emb, dim=1)

                correct += (sim_pos > sim_neg).sum().item()

                del tokenized, generated_ids, generated_texts
                del anchor_emb, positive_emb, negative_emb
                torch.cuda.empty_cache()

    return correct / len(model.dataset)


In [ ]:
evaluate(model)

In [ ]:
!pip install nbstripout

In [ ]:
!nbstripout similarity.ipynb

In [ ]:
!ls ../content